<a href="https://colab.research.google.com/github/sresthabugu/Sentimental-and-toxicity/blob/main/bengxlm1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from transformers import TrainingArguments, Trainer
!pip install evaluate
from datasets import load_dataset,DatasetDict
from transformers import AutoTokenizer,AutoModelForSequenceClassification, DataCollatorWithPadding
import evaluate

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Train.csv to Train (1).csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Val.csv to Val.csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Test.csv to Test.csv


In [ ]:
df_sentnob_train = pd.read_csv("Train.csv")
df_sentnob_val = pd.read_csv("Val.csv")
df_sentnob_test = pd.read_csv("Test.csv")


In [ ]:
df_sentnob_train = df_sentnob_train.sample(frac=1)
df_sentnob_test = df_sentnob_test.sample(frac=1)
df_sentnob_val = df_sentnob_val.sample(frac=1)

In [ ]:
import datasets
from datasets import load_dataset, DatasetDict
sentnob = DatasetDict()
sentnob["train"] = datasets.Dataset.from_pandas(df = df_sentnob_train)
sentnob["test"] = datasets.Dataset.from_pandas(df = df_sentnob_test)
sentnob['val'] = datasets.Dataset.from_pandas(df = df_sentnob_val)
sentnob

DatasetDict({
    train: Dataset({
        features: ['Data', 'Label', '__index_level_0__'],
        num_rows: 12575
    })
    test: Dataset({
        features: ['Data', 'Label', '__index_level_0__'],
        num_rows: 1586
    })
    val: Dataset({
        features: ['Data', 'Label', '__index_level_0__'],
        num_rows: 1567
    })
})

In [ ]:
sentnob = sentnob.rename_column("Label","labels")
sentnob = sentnob.rename_column("Data","text")
sentnob = sentnob.remove_columns("__index_level_0__")
sentnob

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 12575
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 1586
    })
    val: Dataset({
        features: ['text', 'labels'],
        num_rows: 1567
    })
})

In [ ]:
checkpoint = 'FacebookAI/xlm-roberta-base'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 3)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_fn(examples):
    return tokenizer(examples['text'], truncation = 'longest_first')

In [ ]:
tokenized_datasets = sentnob.map(tokenize_fn, batched = True)
tokenized_datasets

Map:   0%|          | 0/12575 [00:00<?, ? examples/s]

Map:   0%|          | 0/1586 [00:00<?, ? examples/s]

Map:   0%|          | 0/1567 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 12575
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1586
    })
    val: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1567
    })
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.combine(["accuracy"])
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [ ]:
!pip install wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.1/302.1 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.login(key='1ed359577e44aad89b7fdd568b565b0c8be5a426')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch", per_device_train_batch_size = 16, per_device_eval_batch_size = 16, num_train_epochs = 5, learning_rate = 2e-5, weight_decay = 0.1)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(model, training_args, train_dataset = tokenized_datasets['train'], eval_dataset = tokenized_datasets['val'], data_collator = data_collator, tokenizer = tokenizer)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: tarik2568 (tarikul_tamiti). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.960100,0.788646
2,0.738200,0.756344
3,0.673200,0.721546
4,0.549700,0.713777
5,0.491900,0.744006


TrainOutput(global_step=3930, training_loss=0.6706589754604505, metrics={'train_runtime': 1603.7124, 'train_samples_per_second': 39.206, 'train_steps_per_second': 2.451, 'total_flos': 2173959711069174.0, 'train_loss': 0.6706589754604505, 'epoch': 5.0})

In [ ]:
predictions = trainer.predict(tokenized_datasets["val"])
ground_truth_labels = predictions.label_ids
predicted_labels = predictions.predictions.argmax(axis=1)  # Extract predicted labels

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report

accuracy = accuracy_score(ground_truth_labels, predicted_labels)
f1_macro = f1_score(ground_truth_labels, predicted_labels, average='macro')
f1_micro = f1_score(ground_truth_labels, predicted_labels, average='micro')

precision = precision_score(ground_truth_labels, predicted_labels, average="weighted")
recall = recall_score(ground_truth_labels, predicted_labels, average="weighted")
class_report = classification_report(ground_truth_labels, predicted_labels)
conf_matrix = confusion_matrix(ground_truth_labels, predicted_labels)

print("Accuracy:", accuracy)
print("F1 Macro:", f1_macro)
print("F1 Micro:", f1_micro)
print('Precision:', precision)
print("Recall:", recall)
print("Classification Report:\n", class_report)
print("Confusion Matrix: \n", conf_matrix)

Accuracy: 0.7128270580727505
F1 Macro: 0.6803970040966946
F1 Micro: 0.7128270580727506
Precision: 0.708364414434686
Recall: 0.7128270580727505
Classification Report:
               precision    recall  f1-score   support

           0       0.52      0.47      0.49       354
           1       0.76      0.77      0.77       623
           2       0.77      0.79      0.78       590

    accuracy                           0.71      1567
   macro avg       0.68      0.68      0.68      1567
weighted avg       0.71      0.71      0.71      1567

Confusion Matrix: 
 [[168  96  90]
 [ 88 482  53]
 [ 70  53 467]]
